## Question 1

#### Grade [radverma](https://github.iu.edu/CSCI-P556-Spring-2021/P556-radverma/tree/master/hw3) submission for hw3

In [ ]:
import math
import numpy as np
from copy import deepcopy

#### Implementation of sigmoid function and derivative of a sigmoid function. Sigmoid function will be used for all activation units. Derivative will be used to calculate weight updates.

In [ ]:
def init_weights():
    return [np.random.uniform(-1, 1, (5, 4))] + [np.random.uniform(-1, 1, (5, 1))]


def to_binary(n):
    return np.unpackbits(np.arange(n, n + 1, dtype='>i%d' % 1).view(np.uint8))[-4:]


# ACTIVATION FUNCTION
def sigmoid(z):
    return 1 / (1 + np.exp(-z))
    # return math.tanh(x)


def train():
    X = np.array([[1] + list(to_binary(i)) for i in range(0, 16)]).T
    D = np.array([bin(i).count("1") % 2 != 0 for i in range(0, 16)]).reshape(1, 16).astype(int)
    return X, D

#### Implementation of forward pass

#### Implementation of backward pass with momentum, where momentum is zero, when momentum is not to be used.

In [ ]:
def forward_propagation(x, layers):
    output = []
    output.append(sigmoid(np.dot(layers[0].T, x)))
    output.append(np.dot(layers[0].T, np.insert(output[0], 0, 1, axis=0)))
    return np.array(output)


def backward_propagation(x, Y, d1, L, eta, delta_l, alpha):
    y = Y[1]
    deltaK = y * (1 - y) * (d1 - y)
    delta_l[1] = eta * deltaK.T * np.insert(Y[0], 0, 1, axis=0) + (alpha * delta_l[1])

    deltaJ = Y[0] * (1 - Y[0]) * L[1][1:, :] * deltaK
    delta_l[0] = eta * deltaJ.T * x + (alpha * delta_l[0])

    L[0] += delta_l[0]
    L[1] = L[1] + delta_l[1]



#### Multi Layer Perceptron

In [ ]:
def MultiLayerPerceptron(X, D, L, eta, alpha):
    epoch = 0
    delta_l = [np.zeros((5, 4))] + [np.zeros((5, 1))]

    while True:
        error = False
        rng = np.random.default_rng()
        stochasticX = rng.permutation(X)
        for i in range(0, 16):
            x = stochasticX[:, i].reshape(5, 1)
            Y = forward_propagation(x, L)
            backward_propagation(x, Y, np.reshape(D[:, i], (1, 1)), L, eta, delta_l, alpha)

            if np.abs(np.squeeze(D[:, i] - Y[1])) < 0.05:
                error = True
            if epoch % 10000 == 0:
                print("Epoch", epoch)
        epoch += 1

        if not error:
            break

    return epoch


In [ ]:
if __name__ == "__main__":
    X, D = train()
    L = init_weights()
    for eta in np.arange(0.05, 0.51, 0.05):
        print("eta : ", eta, ", momentum : ", 0, " epoch : ", MultiLayerPerceptron(X, D, deepcopy(L), eta, 0))
        print("eta : ", eta, ", momentum :", 0.9, " epoch : ", MultiLayerPerceptron(X, D, deepcopy(L), eta, 0.9))
